In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib
import matplotlib.pyplot as plt
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180

import sys
sys.path.append('/home/cat/code/gerbil/utils/')

#
import os
import numpy as np

#
from visualize import Visualize
from track import Track


Autosaving every 180 seconds


2022-11-01 07:56:57.301747: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/cat/miniconda3/envs/gerbils/lib/python3.10/site-packages/cv2/../../lib64:
2022-11-01 07:56:57.301765: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [12]:
######################################################## 
################ GENERATE FIXED TRACK ##################
########################################################

if True:
    # 
    fname = '/media/cat/256GB/dan/testing_track_cleanup_code/2020_07_26_13_02_29_482519_compressed_Day_huddle/2020_07_26_13_02_29_482519_compressed_Day_huddle.slp'
    fname = '/media/cat/256GB/dan/testing_track_cleanup_code/p22_huddles.npy'
    fname = '/home/cat/data/dan/p15_huddles_filled.npy'
    
    track = Track(fname)

    #############################################
    ############# RUN TRACK FIXER ###############
    #############################################
    max_jump_allowed = 50,              # maximum distance that a gerbil can travel in 1 frame
    max_dist_to_join = 50,              # maximum distnace between 2 chunks that can safely be merged
    min_chunk_len = 25

    track.fix_huddles(max_jump_allowed,             
                      max_dist_to_join,            
                      min_chunk_len)

##################################################
################# RUN HUDDLE FIXER ###############
##################################################
if True:

    #
    track.max_distance_huddle = 100                   # how far away we can merge huddles together (pixels)
    track.max_time_to_join_huddle = 25*120            # how far in time can we merge huddle chunks (seconds x frames)
    track.min_huddle_time = 30*25                     # minimum huddle duration in seconds
    track.memory_interpolate_huddle()

    ##################################################
    ############## SAVE FIXED TRACKS #################
    ##################################################

    ##### save the fixed spines will overwrite the previous/defatul spine values####
    track.save_centroid()

    #
    track.save_updated_huddle_tracks()

#
print ("Done...")

spine tracks:  (2160000, 1, 2)
 cleaning tracks spine


setting big jumps to nans: 100%|██| 2159998/2159998 [00:13<00:00, 164567.00it/s]
join segements that are close: 100%|█| 2159998/2159998 [00:02<00:00, 846343.27it
interpolate betweensmall bits: 100%|█| 2159998/2159998 [00:02<00:00, 852749.82it


... Making tracks chunks...


100%|█████████████████████████████| 2159999/2159999 [00:03<00:00, 554128.37it/s]


 cleaning tracks spine


...

running memory interpolation on huddles
 Total # of starting huddle segments:  1795


Remaining segs to process 78:   0%|                    | 0/1795 [00:29<?, ?it/s]


saving centroid
 # of detected tracks/huddles:  12


100%|██████████████████████████████████████████| 12/12 [00:00<00:00, 299.45it/s]


fname out:  /home/cat/data/dan/p15_huddles_filled_multi_track_huddles.npy
Done...


In [13]:
#########################################################
############### LOAD THE MULTI HUDDLE TRACK #############
#########################################################
#
#fname_slp = '/media/cat/256GB/dan/testing_track_cleanup_code/2020_07_26_18_35_09_660833_compressed_Day_huddle/2020_07_26_18_35_09_660833_compressed_Day_huddle.slp'
#data = np.load(fname_slp.replace('.slp', '_multi_track_huddles.npy'))
fname = '/home/cat/data/dan/p15_huddles_filled_multi_track_huddles.npy'
data = np.load(fname)
print (data.shape)

fps = 25
total_time = data.shape[0]/fps

#
plt.figure()
for k in range(data.shape[1]):
    
    seg = data[:,k]
    
    #
    idx = np.where(np.isnan(seg[:,0])==False)[0]
    time = idx.shape[0]/fps
    huddle_percent = time/total_time*100
    #print (k, time, "sec", " # of pts to plot: ", seg.shape[0])
    
    #
    if True:
        plt.plot(seg[idx,0],
             seg[idx,1],
             label='huddle #: '+str(k)+
             ", duration: "+str(time)+ " , "+
             ", start hour: "+ str(round(idx[0]/2160000*24,2)) +
             ", % of rec: " + str(round(huddle_percent,2))+ " %")   
        
    #if k>30:
    #    break
#
plt.xlim(0,900)
plt.ylim(0,700)
plt.legend(fontsize=6, ncol=2)
plt.suptitle(os.path.split(fname)[1], fontsize=12)
plt.show()


(2160000, 12, 2)


In [15]:
fname = '/home/cat/Downloads/Downloads/predictions_20221018104023.parquet'
import pandas as pd
data = pd.read_parquet(fname, engine='fastparquet')

data

,VIDEO,FRAME,animal_1_x,animal_1_y,animal_2_x,animal_2_y,0,1,2,3,...,29,30,31,32,33,34,35,36,37,38
0,2020_08_01_10_18_35_455807.npy,0,183.873978,531.765564,132.432659,192.133110,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
1,2020_08_01_10_18_35_455807.npy,1,183.829651,531.750549,132.597404,196.190704,0.046801,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
2,2020_08_01_10_18_35_455807.npy,2,183.904083,531.736633,136.091064,198.015427,0.075722,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
3,2020_08_01_10_18_35_455807.npy,3,183.891312,531.739136,136.016304,196.332924,0.013014,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
4,2020_08_01_10_18_35_455807.npy,4,183.856094,531.740540,136.037605,196.337395,0.035245,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28797,2020_07_30_15_43_41_766841.npy,28797,416.585815,367.319550,549.843262,542.037109,0.420915,2.334314,3.607108,2.653973,...,-9.740968,1.557563,-4.047885,0.0,0.0,0.0,0.924426,0.355451,-1.635742,1.563190
28798,2020_07_30_15_43_41_766841.npy,28798,419.261017,367.689362,550.035278,542.205017,2.700641,1.524737,3.613697,2.532629,...,-9.216387,0.522876,-0.143916,0.0,0.0,0.0,0.810425,0.063709,-1.543547,1.605570
28799,2020_07_30_15_43_41_766841.npy,28799,416.125732,368.047119,549.751160,541.981781,3.155630,1.495954,3.799507,2.520698,...,-9.068857,-0.480154,-0.522175,0.0,0.0,0.0,0.580109,-0.013916,-1.784398,1.613001
28800,2020_07_30_15_43_41_766841.npy,28800,412.588715,368.448822,549.937195,535.931671,3.559756,1.391519,3.738917,2.532746,...,-8.845525,-1.513044,3.240859,0.0,0.0,0.0,-0.250117,0.216321,-1.987061,1.625117


In [ ]:
## OLD CODE BELOW ##

### The code above outputs 2 lists of lists
track.final_merged_times
- this contains all the time segments that were merged into a single array plus the interpolated times

track.final_merged_locs
- this contains the locations of the animals from each of these lists

There can be several huddles, depending on how far away we merge them
- recommended distance is 100 pixels


In [4]:
####################################################
########## MAKE PLOTS OF SPINE CENTRES #############
####################################################
    
#
print ("# of huddles: ", len(track.final_merged_times))

#
times = []
for k in range(len(track.final_merged_times)):
    segs = track.final_merged_times[k]
    
    total_time = 0
    for seg in segs:
        #print (k, seg)
        total_time+=seg[1]-seg[0]
    
    times.append(total_time)

print ("times: ", times)
    
    
#############################################################
###################### PLOT THE HUDDLES #####################
#############################################################
plt.figure()
colors = plt.cm.rainbow(np.linspace(0,1,len(track.final_merged_times)))
for k in range(len(track.final_merged_times)):
    segs = track.final_merged_locs[k]
    
    #
    print ("# of huddle segs : ", len(segs))
    
    segs = np.vstack(segs)
    print (segs.shape)
    
    #
    plt.plot(segs[:,0], 
             segs[:,1],
             color=colors[k],
             label='huddle #' + str(k) + ", time: "+str(segs.shape[0]/25)+ "sec" + ", "+
             str(int(segs.shape[0]/track.tracks.shape[0]*100))+"%")
   
        
#
plt.legend()

#        
plt.xlim(0,1000)
plt.ylim(0,800)
#break
plt.show()

# of huddles:  4
times:  [25011, 29, 10412, 5762]
# of huddle segs :  203
(25011, 2)
# of huddle segs :  3
(29, 2)
# of huddle segs :  105
(10412, 2)
# of huddle segs :  9
(5762, 2)


In [25]:
coords = np.random.rand(2,10).T
print (coords.shape)
dist = np.array([np.linalg.norm(v) for v in coords])

print (dist)
    
##### save the fixed spines will overwrite the previous/defatul spine values####
dist = np.linalg.norm(coords,axis=1)
print (dist)


(10, 2)
[0.75475524 0.83475646 0.84746844 0.9678204  0.62845943 1.29847907
 0.99270843 0.79906404 1.02981141 0.66171543]
[0.75475524 0.83475646 0.84746844 0.9678204  0.62845943 1.29847907
 0.99270843 0.79906404 1.02981141 0.66171543]
